In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import StandardScaler
import autogluon
from autogluon.tabular import TabularDataset, TabularPredictor

/Users/olabub/Desktop/SUML/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [5]:
# usuniecie kolumn niepotrzebnych i mocno skorelowanych
train = train.drop(['id', 'Unnamed: 0', 'Arrival Delay in Minutes','Ease of Online booking'], axis=1)
test = test.drop(['id', 'Unnamed: 0', 'Arrival Delay in Minutes','Ease of Online booking'], axis=1)

In [6]:
#sprawdzenie czy są wartości odstające train
numeric_train = train.select_dtypes(include='number')

for column in numeric_train.columns:
  Q1 = numeric_train[column].quantile(0.25)
  Q3 = numeric_train[column].quantile(0.75)
  IQR = Q3 - Q1
  outliers = numeric_train[(numeric_train[column] < (Q1 - 1.5 * IQR)) | (numeric_train[column] > (Q3 + 1.5 * IQR))]
  if not outliers.empty:
    print(f'W kolumnie {column} są wartości odstające.')

#usunięcie wartości odstających train
train_no_outliers = train.copy()
for column in train_no_outliers.select_dtypes(include='number').columns:
    Q1 = train_no_outliers[column].quantile(0.25)
    Q3 = train_no_outliers[column].quantile(0.75)
    IQR = Q3 - Q1
    train_no_outliers = train_no_outliers[(train_no_outliers[column] >= (Q1 - 1.5 * IQR)) &
                                              (train_no_outliers[column] <= (Q3 + 1.5 * IQR))]

#normalizacja danych
numeric_columns = train_no_outliers.select_dtypes(include='number').columns
scaler = StandardScaler()

train_no_outliers[numeric_columns] = scaler.fit_transform(train_no_outliers[numeric_columns])

print(train_no_outliers)

#podział na x i y
x_train = train_no_outliers.drop("satisfaction", axis=1)
y_train = train_no_outliers["satisfaction"].copy()


W kolumnie Flight Distance są wartości odstające.
W kolumnie Checkin service są wartości odstające.
W kolumnie Departure Delay in Minutes są wartości odstające.
        Gender      Customer Type       Age   Type of Travel     Class  \
0         Male     Loyal Customer -1.751975  Personal Travel  Eco Plus   
2       Female     Loyal Customer -0.891052  Business travel  Business   
4         Male     Loyal Customer  1.426820  Business travel  Business   
5       Female     Loyal Customer -0.891052  Personal Travel       Eco   
6         Male     Loyal Customer  0.499671  Personal Travel       Eco   
...        ...                ...       ...              ...       ...   
103899  Female  disloyal Customer -1.089726  Business travel       Eco   
103900    Male     Loyal Customer  0.632121  Business travel  Business   
103901    Male  disloyal Customer -0.626152  Business travel  Business   
103902  Female  disloyal Customer -1.155951  Business travel       Eco   
103903    Male     Loyal 

In [7]:
#sprawdzenie czy są wartości odstające test
numeric_test = test.select_dtypes(include='number')

for column in numeric_test.columns:
  Q1 = numeric_test[column].quantile(0.25)
  Q3 = numeric_test[column].quantile(0.75)
  IQR = Q3 - Q1
  outliers = numeric_test[(numeric_test[column] < (Q1 - 1.5 * IQR)) | (numeric_test[column] > (Q3 + 1.5 * IQR))]
  if not outliers.empty:
    print(f'W kolumnie {column} są wartości odstające.')

#usunięcie wartości odstających train
test_no_outliers = test.copy()
for column in test_no_outliers.select_dtypes(include='number').columns:
    Q1 = test_no_outliers[column].quantile(0.25)
    Q3 = test_no_outliers[column].quantile(0.75)
    IQR = Q3 - Q1
    test_no_outliers = test_no_outliers[(test_no_outliers[column] >= (Q1 - 1.5 * IQR)) &
                                              (test_no_outliers[column] <= (Q3 + 1.5 * IQR))]

#normalizacja danych
numeric_columns = test_no_outliers.select_dtypes(include='number').columns
scaler = StandardScaler()

test_no_outliers[numeric_columns] = scaler.fit_transform(test_no_outliers[numeric_columns])

print(test_no_outliers)
#test_no_outliers.drop("satisfaction", axis=1)
#podział na x i y
# x_test = test.drop("satisfaction", axis=1)
# y_test = test["satisfaction"].copy()

W kolumnie Flight Distance są wartości odstające.
W kolumnie Checkin service są wartości odstające.
W kolumnie Departure Delay in Minutes są wartości odstające.
       Gender      Customer Type       Age   Type of Travel     Class  \
1      Female     Loyal Customer -0.241529  Business travel  Business   
2        Male  disloyal Customer -1.296114  Business travel       Eco   
3        Male     Loyal Customer  0.285763  Business travel  Business   
4      Female     Loyal Customer  0.615321  Business travel       Eco   
8        Male     Loyal Customer  0.483498  Business travel       Eco   
...       ...                ...       ...              ...       ...   
25970  Female  disloyal Customer -0.241529  Business travel       Eco   
25971    Male  disloyal Customer -0.373352  Business travel  Business   
25972    Male     Loyal Customer -1.098379  Business travel  Business   
25973  Female     Loyal Customer -1.493849  Personal Travel       Eco   
25974    Male     Loyal Customer -1.

In [9]:
# na wyczyszczonych
from sklearn.metrics import classification_report, confusion_matrix

predictor = TabularPredictor(label='satisfaction').fit(
    train_no_outliers,
    presets="good",
    time_limit=200,
    dynamic_stacking=False,
    num_stack_levels=0,
    ag_args_fit={'random_seed': 42}
)

predictor.save()
predictions = predictor.predict(test_no_outliers)

leaderboard = predictor.leaderboard()
print(leaderboard)
print(predictor.evaluate(test_no_outliers))

y_true = test_no_outliers["satisfaction"]
X_test = test_no_outliers.drop(columns=["satisfaction"])

y_pred = predictor.predict(X_test)

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))

model_name=predictor.model_best
model_info=predictor.info()
#model_info['model_info']['model_name']['hyperparameters']
print(model_name)
print(model_info)

No path specified. Models will be saved in: "AutogluonModels/ag-20260116_130919"
Preset alias specified: 'good' maps to 'good_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.9.6
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.6.0: Mon Jul 14 11:29:54 PDT 2025; root:xnu-11417.140.69~1/RELEASE_ARM64_T8122
CPU Count:          8
Memory Avail:       1.26 GB / 8.00 GB (15.8%)
Disk Space Avail:   49.48 GB / 228.27 GB (21.7%)
Presets specified: ['good']
Using hyperparameters preset: hyperparameters='light'
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag_folds=True`.
B

                          model  score_val eval_metric  pred_time_val  \
0           WeightedEnsemble_L2   0.959152    accuracy       2.116791   
1       RandomForestGini_BAG_L1   0.958774    accuracy       1.055264   
2       RandomForestEntr_BAG_L1   0.958656    accuracy       1.059882   
3         ExtraTreesEntr_BAG_L1   0.952804    accuracy       1.129046   
4         ExtraTreesGini_BAG_L1   0.952309    accuracy       1.144808   
5  RandomForestGini_BAG_L1_FULL        NaN    accuracy       1.055264   
6  RandomForestEntr_BAG_L1_FULL        NaN    accuracy       1.059882   
7    ExtraTreesEntr_BAG_L1_FULL        NaN    accuracy       1.129046   
8    ExtraTreesGini_BAG_L1_FULL        NaN    accuracy       1.144808   
9      WeightedEnsemble_L2_FULL        NaN    accuracy            NaN   

   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  \
0  6.273589                0.001645           0.093345            2   
1  4.139444                1.055264           4.139444

In [12]:
# LICZENIE WAŻNOŚCI CECH
fi = predictor.feature_importance(test_no_outliers)
fi


Computing feature importance via permutation shuffling for 20 features using 5000 rows with 5 shuffle sets...
	23.28s	= Expected runtime (4.66s per shuffle set)
	4.65s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
Type of Travel,0.14256,0.003678,5.312175e-08,5,0.150133,0.134987
Inflight wifi service,0.12468,0.006715,1.005808e-06,5,0.138506,0.110854
Customer Type,0.05200,0.004915,9.466851e-06,5,0.062121,0.041879
Online boarding,0.03684,0.004281,2.149578e-05,5,0.045655,0.028025
Checkin service,0.02180,0.003212,5.498493e-05,5,0.028415,0.015185
Class,0.01748,0.001820,1.389778e-05,5,0.021227,0.013733
Seat comfort,0.01376,0.002582,1.420941e-04,5,0.019077,0.008443
Cleanliness,0.01092,0.002689,4.078312e-04,5,0.016457,0.005383
Baggage handling,0.01048,0.001368,3.408333e-05,5,0.013297,0.007663
Inflight service,0.00564,0.001977,1.548755e-03,5,0.009710,0.001570


In [13]:
top_features = fi.sort_values("importance", ascending=False).head(8)
print(top_features)
selected_features = top_features.index.tolist()
print(selected_features)


                       importance    stddev       p_value  n  p99_high  \
Type of Travel            0.14256  0.003678  5.312175e-08  5  0.150133   
Inflight wifi service     0.12468  0.006715  1.005808e-06  5  0.138506   
Customer Type             0.05200  0.004915  9.466851e-06  5  0.062121   
Online boarding           0.03684  0.004281  2.149578e-05  5  0.045655   
Checkin service           0.02180  0.003212  5.498493e-05  5  0.028415   
Class                     0.01748  0.001820  1.389778e-05  5  0.021227   
Seat comfort              0.01376  0.002582  1.420941e-04  5  0.019077   
Cleanliness               0.01092  0.002689  4.078312e-04  5  0.016457   

                        p99_low  
Type of Travel         0.134987  
Inflight wifi service  0.110854  
Customer Type          0.041879  
Online boarding        0.028025  
Checkin service        0.015185  
Class                  0.013733  
Seat comfort           0.008443  
Cleanliness            0.005383  
['Type of Travel', 'Infligh

In [15]:
#WYBRANE NAJWAŻNIEJSZE CECHY --> NOWE ZBIORY
features = [
    'Type of Travel',
    'Inflight wifi service',
    'Customer Type',
    'Online boarding',
    'Checkin service',
    'Seat comfort',
    'Class',
    'Cleanliness',
    'satisfaction'
]
train_new=train_no_outliers[features]
test_new = test_no_outliers[features]
print(train_new)
print(test_new)

         Type of Travel  Inflight wifi service      Customer Type  \
0       Personal Travel               0.184799     Loyal Customer   
2       Business travel              -0.558651     Loyal Customer   
4       Business travel               0.184799     Loyal Customer   
5       Personal Travel               0.184799     Loyal Customer   
6       Personal Travel              -0.558651     Loyal Customer   
...                 ...                    ...                ...   
103899  Business travel              -0.558651  disloyal Customer   
103900  Business travel               0.928249     Loyal Customer   
103901  Business travel              -1.302100  disloyal Customer   
103902  Business travel              -1.302100  disloyal Customer   
103903  Business travel              -1.302100     Loyal Customer   

        Online boarding  Checkin service  Seat comfort     Class  Cleanliness  \
0             -0.230028         0.369770      1.142986  Eco Plus     1.277367   
2        

In [17]:
#MODEL DO WYTRENOWANIA TYLKO NA NAJWAŻNIEJSZYCH CECHACH
from sklearn.metrics import classification_report, confusion_matrix

predictor = TabularPredictor(label='satisfaction', path='features_10_400/').fit(
    train_new,
    presets="good",
    time_limit=400,
    dynamic_stacking=False,
    num_stack_levels=0,
    ag_args_fit={'random_seed': 42}
)
predictor.save('features_10_400/')
predictions = predictor.predict(test_new)

leaderboard = predictor.leaderboard()
print(leaderboard)
print(predictor.evaluate(test_new))

y_true = test_new["satisfaction"]
X_test = test_new.drop(columns=["satisfaction"])

y_pred = predictor.predict(X_test)

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))

model_name=predictor.model_best
model_info=predictor.info()
#model_info['model_info']['model_name']['hyperparameters']
print(model_name)
print(model_info)

Preset alias specified: 'good' maps to 'good_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.9.6
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.6.0: Mon Jul 14 11:29:54 PDT 2025; root:xnu-11417.140.69~1/RELEASE_ARM64_T8122
CPU Count:          8
Memory Avail:       1.40 GB / 8.00 GB (17.5%)
Disk Space Avail:   48.69 GB / 228.27 GB (21.3%)
Presets specified: ['good']
Using hyperparameters preset: hyperparameters='light'
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag_folds=True`.
Beginning AutoGluon training ... Time limit = 400s
AutoGluon will save models to "

                          model  score_val eval_metric  pred_time_val  \
0           WeightedEnsemble_L2   0.940174    accuracy       2.001980   
1         ExtraTreesGini_BAG_L1   0.940109    accuracy       0.998318   
2         ExtraTreesEntr_BAG_L1   0.939796    accuracy       1.001457   
3       RandomForestEntr_BAG_L1   0.939523    accuracy       0.958179   
4       RandomForestGini_BAG_L1   0.939314    accuracy       0.951001   
5  RandomForestGini_BAG_L1_FULL        NaN    accuracy       0.951001   
6  RandomForestEntr_BAG_L1_FULL        NaN    accuracy       0.958179   
7    ExtraTreesGini_BAG_L1_FULL        NaN    accuracy       0.998318   
8    ExtraTreesEntr_BAG_L1_FULL        NaN    accuracy       1.001457   
9      WeightedEnsemble_L2_FULL        NaN    accuracy            NaN   

   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  \
0  1.716362                0.002205           0.092237            2   
1  0.805925                0.998318           0.805925

In [19]:
train_new.to_csv("data/train_new.csv", index=False)
test_new.to_csv("data/test_new.csv", index=False)


In [25]:
# LICZENIE WAŻNOŚCI CECH
fi = predictor.feature_importance(test_new)
fi


Computing feature importance via permutation shuffling for 8 features using 5000 rows with 5 shuffle sets...
	10.46s	= Expected runtime (2.09s per shuffle set)
	2.78s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
Type of Travel,0.16596,0.003170,1.596304e-08,5,0.172487,0.159433
Inflight wifi service,0.13780,0.007870,1.271277e-06,5,0.154005,0.121595
Customer Type,0.07776,0.002674,1.674330e-07,5,0.083265,0.072255
Online boarding,0.04776,0.006673,4.456257e-05,5,0.061500,0.034020
Seat comfort,0.04140,0.006431,6.768543e-05,5,0.054642,0.028158
Cleanliness,0.03928,0.004668,2.349389e-05,5,0.048892,0.029668
Checkin service,0.03180,0.001691,9.562444e-07,5,0.035282,0.028318
Class,0.02216,0.001466,2.282665e-06,5,0.025178,0.019142
